# Database Engineering

All database engineering will be completed within this notebook. SQLAlchemy will be utilized to model table schemas and create a sqlite database for the created tables. 

In [5]:
# Read in dependencies
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [2]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
# Create a connection to the engine called `conn`
conn = engine.connect()

### Creating the Measurements Table

In [7]:
# Use `declarative_base` from SQLAlchemy to model the measurements table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    precip = Column(Float)  
    tobs = Column(Float)

In [8]:
# Use `create_all` to create the measurements table in the database
Base.metadata.create_all(engine)

In [10]:
# Load the cleaned csv file into a pandas dataframe
measurements_df = pd.read_csv("Resources/clean_hawaii_measurements.csv")
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,NaN,73


### Creating the Stations Table

In [ ]:
# Use `declarative_base` from SQLAlchemy to model the measurements table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
class Stations(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    precip = Column(Float)  
    tobs = Column(Float)